# RecSys Project - Group 33

In this part of the project, we would like to try to implement a recommendation model form __TensorFlow Recommenders__: https://www.tensorflow.org/recommenders/examples/quickstart.

Here we implement the basic retrieval model, which takes session data, pads it with zeros until the maximum session length, further transforms it into an emdedded representation of 32 values and fits it to the GRU layer. This way we receive a query representation, which is matched by dot product with a candidate representation, consisting of 32 values as well.  
In this basic scenario for the __query model__ we take items in the session and for the __candidate model__ we take the purchsed item representation. 

The MRR for the basic model is 0.01. 

This model can be __improved__ by adding information about _timestamps_ and _item features_ to the query model and to the candidate model at the same time.  We can do it by normalizing timestamps and bucketing them. Item features can be also emdedded by concidering "category_description" as one categorical feature.

Further the information from both models can by separately concatenated and fed into a dense layer to receive an equally-sized representations for a query and a candidate model. 


## 1 | Libaries & Data

Import all libraries and the data

In [53]:
#pip install tensorflow_recommenders
#!pip install tensorflow_ranking

In [54]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import random
import logging
import pandas as pd
import tensorflow_ranking as tfr

### Load sessions data

In [5]:
directory = "~/shared/data/project/training/"

In [6]:
DATA_NAME = "train_sessions.csv"
train_sessions = pd.read_csv(directory + DATA_NAME,parse_dates=['date'],dtype={
                     'session_id': int,
                     'item_id': int
                 })
train_sessions['timestemp'] = train_sessions['date'].values.astype('datetime64[s]').astype(np.int64) # to_unix/to_timestemp



In [7]:
train_sessions.dtypes

session_id             int64
item_id                int64
date          datetime64[ns]
timestemp              int64
dtype: object

In [8]:
# Drop columns, group by session_id, and pad with 0 - length = max length
train_sessions = train_sessions[['session_id', 'item_id','timestemp' ]]
unique_item_ids = np.unique(train_sessions['item_id'])
train_sessions = train_sessions[['session_id', 'item_id','timestemp' ]].groupby('session_id').agg(list).reset_index(level=0)

# Pad with 0
max_len = max([len(line) for line in train_sessions.item_id])
[ line.extend([0]*(max_len-len(line))) for line in train_sessions.item_id if len(line)<max_len]
[ line.extend([0]*(max_len-len(line))) for line in train_sessions.timestemp if len(line)<max_len]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

### Load Purchase data

In [16]:
DATA_NAME = "train_purchases.csv"
col_names=['session_id_p', 'item_id_p', 'date_p']
train_purchases = pd.read_csv(directory+DATA_NAME,parse_dates=['date'],dtype={
                     'session_id': int,
                     'item_id': int
                 })
train_purchases['timestemp'] = train_purchases['date'].values.astype('datetime64[s]').astype(np.int64) # to_unix/to_timestemp
unique_item_ids_p = np.unique(train_purchases['item_id'])

In [17]:
train_df = train_sessions.merge(train_purchases, on='session_id', how='inner', suffixes=('_s', '_p'))
train_df.drop('date',axis = 1, inplace  = True)
train_df.rename(columns = { 'item_id_s':'item_id', 'timestemp_s': 'timestemp', 'item_id_p':'label'}, inplace = True)
train_df.drop('timestemp_p',axis = 1, inplace  = True)
train_df.head()

,session_id,item_id,timestemp,label
0,3,"[9655, 9655, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1608326700, 1608326388, 0, 0, 0, 0, 0, 0, 0, ...",15085
1,13,"[15654, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1584128127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",18626
2,18,"[18316, 2507, 4026, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1598469510, 1598469391, 1598469347, 0, 0, 0, ...",24911
3,19,"[25772, 6341, 25555, 20033, 8281, 8268, 4385, ...","[1604334678, 1604334873, 1604335384, 160433532...",12534
4,24,"[2927, 11662, 2927, 28075, 434, 16064, 10414, ...","[1582737784, 1582737989, 1582737768, 158274135...",13226


In [12]:
train_df.head()

,session_id,item_id_s,timestemp_s,item_id_p,timestemp_p
0,3,"[9655, 9655, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1608326700, 1608326388, 0, 0, 0, 0, 0, 0, 0, ...",15085,1608326807
1,13,"[15654, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1584128127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",18626,1584128175
2,18,"[18316, 2507, 4026, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1598469510, 1598469391, 1598469347, 0, 0, 0, ...",24911,1598469632
3,19,"[25772, 6341, 25555, 20033, 8281, 8268, 4385, ...","[1604334678, 1604334873, 1604335384, 160433532...",12534,1604337405
4,24,"[2927, 11662, 2927, 28075, 434, 16064, 10414, ...","[1582737784, 1582737989, 1582737768, 158274135...",13226,1582741664


In [18]:
from tqdm import tqdm

## 2 | Prepare datasets

In [19]:
examples = []
for index,row in tqdm(train_df.iterrows()):
    item_id = [x for x in row['item_id']]
    timestemp = [x for x in row['timestemp']]
    label = row['label']
    feature = {"context_item_id":
           tf.train.Feature(
               int64_list=tf.train.Int64List(value= item_id)),
           "context_item_timestemp":
           tf.train.Feature(
               int64_list=tf.train.Int64List(value= timestemp)),
               "label":
           tf.train.Feature(
               int64_list=tf.train.Int64List(value= [label])),
            }
    tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
    examples.append(tf_example)

1000000it [01:33, 10679.61it/s]


### Generate datasets and do the train|test split 90%

In [25]:
OUTPUT_TRAINING_DATA_FILENAME = 'train.tfrecord'
OUTPUT_TESTING_DATA_FILENAME = 'test.tfrecord'

def generate_train_test(examples, train_data_fraction=0.9,random_seed = 123, shuffle = True):
    if shuffle:
        random.seed(random_seed)
        random.shuffle(examples)
    last_train_index = round(len(examples) * train_data_fraction)
    train_examples = examples[:last_train_index]
    test_examples = examples[last_train_index:]
    return train_examples, test_examples



def write_tfrecords(tf_examples, filename):
  """Writes tf examples to tfrecord file, and returns the count."""
  with tf.io.TFRecordWriter(filename) as file_writer:
    length = len(tf_examples)
    progress_bar = tf.keras.utils.Progbar(length)
    for example in tf_examples:
      file_writer.write(example.SerializeToString())
      progress_bar.add(1)
    return length



def generate_datasets(examples, 
                      random_seed = 123,
                      shuffle = True,
                      train_data_fraction=0.9,
                      train_filename=OUTPUT_TRAINING_DATA_FILENAME,
                      test_filename=OUTPUT_TESTING_DATA_FILENAME
                     ):
    
    train_examples, test_examples = generate_train_test(examples,
                                                        train_data_fraction=train_data_fraction,
                                                        random_seed = random_seed,
                                                        shuffle = shuffle)

    logging.info("Writing generated training examples.")
    train_file = train_filename
    train_size = write_tfrecords(tf_examples=train_examples, filename=train_file)
    logging.info("Writing generated testing examples.")
    test_file = test_filename
    test_size = write_tfrecords(tf_examples=test_examples, filename=test_file)
    stats = {
          "train_size": train_size,
          "test_size": test_size,
          "train_file": train_file,
          "test_file": test_file,
      }
        
    return stats

In [26]:
stats = generate_datasets(examples)
logging.info("Generated dataset: %s", stats)

100000/100000 [==============================] - 1s 6us/step


In [40]:
train_filename = "train.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "test.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

In [41]:
feature_description = {
    'context_item_id': tf.io.FixedLenFeature([100], tf.int64, default_value=np.repeat(0, 100)),
    'context_item_timestemp': tf.io.FixedLenFeature([100], tf.int64, default_value=np.repeat(0, 100)),
    'label': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    "context_item_id": tf.strings.as_string(x["context_item_id"]),
    #"context_item_timestemp": (x["context_item_timestemp"]),
    "label": tf.strings.as_string(x["label"])
})

test_ds = test.map(_parse_function).map(lambda x: {
    "context_item_id": tf.strings.as_string(x["context_item_id"]),
    #"context_item_timestemp": (x["context_item_timestemp"]),
    "label": tf.strings.as_string(x["label"])
})


In [42]:
for x in train_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'context_item_id': array([b'20754', b'10343', b'21564', b'21564', b'5890', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
       b'0', b'0', b'0', b'0'], dtype=object),
 'label': array([b'24820'], dtype=object)}


In [43]:
# Find unique item_ids for the vocabulary
unique_item_ids_p = np.unique(train_purchases['item_id']).tolist()
unique_item_ids = unique_item_ids.tolist()
unique_item_ids.extend(unique_item_ids_p)
unique_item_ids = list(set(unique_item_ids))
unique_item_ids = np.array(unique_item_ids).astype(str)

## 3 | Define the model

In [55]:
embedding_dimension = 32

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_item_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_item_ids) + 1, embedding_dimension), 
    tf.keras.layers.GRU(embedding_dimension)
])

candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_item_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_item_ids) + 1, embedding_dimension)
])


task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    candidates=item_ids.batch(128).map(candidate_model),
    metrics=[
        tfr.keras.metrics.MRRMetric(),
        tf.keras.metrics.TopKCategoricalAccuracy()
    ],
  )
)

In [45]:
item_ids = tf.data.Dataset.from_tensor_slices(unique_item_ids)

In [56]:
class Model(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task

    def compute_loss(self, features, training=False):
        watch_history = features["context_item_id"]
        watch_next_label = features["label"]

        query_embedding = self._query_model(watch_history)       
        candidate_embedding = self._candidate_model(watch_next_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics = not training)

## 4 | Training the Model

In [57]:
model = Model(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train_ds.batch(4096).cache()
cached_test = test_ds.batch(512).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
220/220 [==============================] - 104s 452ms/step - mrr_metric: 0.0000e+00 - top_k_categorical_accuracy: 0.0000e+00 - loss: 33976.6556 - regularization_loss: 0.0000e+00 - total_loss: 33976.6556
Epoch 2/3
220/220 [==============================] - 88s 400ms/step - mrr_metric: 0.0000e+00 - top_k_categorical_accuracy: 0.0000e+00 - loss: 33976.6435 - regularization_loss: 0.0000e+00 - total_loss: 33976.6435
Epoch 3/3
220/220 [==============================] - 90s 408ms/step - mrr_metric: 0.0000e+00 - top_k_categorical_accuracy: 0.0000e+00 - loss: 33976.6433 - regularization_loss: 0.0000e+00 - total_loss: 33976.6433


## 5 | Evaluating the model

In [59]:
model.evaluate(cached_test, return_dict=True)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
196/196 [==============================] - 63s 313ms/step - mrr_metric: 0.0102 - top_k_categorical_accuracy: 3.2000e-04 - loss: 3169.8395 - regularization_loss: 0.0000e+00 - total_loss: 3169.8395


{'mrr_metric': 0.01015275064855814,
 'top_k_categorical_accuracy': 0.00031999999191612005,
 'loss': 812.02783203125,
 'regularization_loss': 0,
 'total_loss': 812.02783203125}